# A model to predict general things

## Importing the Libraries

In [19]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [20]:
tf.__version__

'2.12.0'

## Data Pre-processing

### Preprocessing the training data

In [21]:
train_data_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

training_set = train_data_generator.flow_from_directory(
    'Data/seg_train/seg_train/',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 14034 images belonging to 6 classes.


In [22]:
training_set.class_indices

{'buildings': 0,
 'forest': 1,
 'glacier': 2,
 'mountain': 3,
 'sea': 4,
 'street': 5}

### Preprocessing the test data

In [23]:
test_data_generator = ImageDataGenerator(rescale=1./255)
testing_set = test_data_generator.flow_from_directory(
    'Data/seg_test/seg_test/',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

Found 3000 images belonging to 6 classes.


In [24]:
testing_set.class_indices

{'buildings': 0,
 'forest': 1,
 'glacier': 2,
 'mountain': 3,
 'sea': 4,
 'street': 5}

## Building the Neural Network

In [25]:
cnn = tf.keras.models.Sequential()

### Adding the convolution layer

In [26]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Adding the pooling layer

In [27]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

### Adding a second convolution layer

In [28]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

### Adding the flattening later

In [29]:
cnn.add(tf.keras.layers.Flatten())

### Full Connection

In [30]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [31]:
cnn.add(tf.keras.layers.Flatten())

### Output Layer

In [32]:
cnn.add(tf.keras.layers.Dense(units=6, activation='softmax'))

## Training the Neural Network

### Compiling the CNN

In [35]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Training the CNN on training set

In [36]:
cnn.fit(x=training_set, validation_data=testing_set, epochs=10)

Epoch 1/10
439/439 [==============================] - 207s 471ms/step - loss: 1.0019 - accuracy: 0.6124 - val_loss: 0.8161 - val_accuracy: 0.6977
Epoch 2/10
439/439 [==============================] - 51s 117ms/step - loss: 0.7733 - accuracy: 0.7109 - val_loss: 0.7672 - val_accuracy: 0.7327
Epoch 3/10
439/439 [==============================] - 51s 116ms/step - loss: 0.6712 - accuracy: 0.7579 - val_loss: 0.7203 - val_accuracy: 0.7497
Epoch 4/10
439/439 [==============================] - 47s 108ms/step - loss: 0.6007 - accuracy: 0.7847 - val_loss: 0.5732 - val_accuracy: 0.7957
Epoch 5/10
439/439 [==============================] - 47s 107ms/step - loss: 0.5506 - accuracy: 0.8012 - val_loss: 0.6354 - val_accuracy: 0.7773
Epoch 6/10
439/439 [==============================] - 50s 115ms/step - loss: 0.5158 - accuracy: 0.8149 - val_loss: 0.6341 - val_accuracy: 0.7853
Epoch 7/10
439/439 [==============================] - 51s 115ms/step - loss: 0.4860 - accuracy: 0.8246 - val_loss: 0.5409 - val_a

In [42]:
import os
import pandas as pd
import numpy as np


folder_path = './Data/seg_pred/seg_pred/'
predictions = []

for filename in os.listdir(folder_path):
    if filename.endswith('.jpg'):  # Adjust the file extension if necessary
        image_path = os.path.join(folder_path, filename)
        test_image = tf.keras.utils.load_img(image_path, target_size=(64, 64))
        test_image = np.array(test_image)
        test_image = np.expand_dims(test_image, axis=0)
        result = cnn.predict(test_image)
        predicted_class = np.argmax(result)
        predictions.append([filename, predicted_class])

df = pd.DataFrame(predictions, columns=['Image', 'Predicted Class'])

# Save the predictions to a CSV file
output_file = 'predictions.csv'
df.to_csv(output_file, index=False)

print(f"Predictions saved to {output_file}.")

1/1 [==============================] - 0s 17ms/step
Predictions saved to predictions.csv.
